# Part 1
## Introduction 
Two point sets (a and b) consisting of 5 points each (5 x 3 matrices) are provided. The task is to find the optimal rotation matrix U such that the RMSD is minimized as well as to calculate the minimum RMSD using both the formula and manually using the rotated coordinates to confirm that similar results are obtained. 

This is achieved by centering both a and b and then calculating the R matrix. Singular Value Decomposition is then performed on the R matrix to obtain the V matrix, S array and transposed W matrix. The rotation matrix U is calculated by multiplying the W matrix with the transposed V matrix. The determinant of the rotation matrix is then calculated. If the determinent of U is -1, the rotation matrix is a roto-reflection which cannot occur in real proteins, so the matrix U is reflected in order to account for this. Finally, the RMSD is calculated using the formula or the matrix b is rotated using the rotation matrix U, and the RMSD of a and the rotated b is calculated.

The following code performs this function:

In [4]:
import numpy as np

def RMSD(a, b, manual = False):
    '''This function takes in two matrices, each containing all the
    coordinates of a protein and returns the transformation matrix
    and the RMSD value'''
    #center a and b
    center_of_mass_a = a.sum(1)/a.shape[1]
    centered_a = a - center_of_mass_a

    center_of_mass_b = b.sum(1)/b.shape[1]
    centered_b = b - center_of_mass_b
    
    #calculate n
    n = np.size(centered_a)/len(centered_a)

    #calculate R
    R = centered_b * centered_a.T

    #SVD
    V, S, W_t = np.linalg.svd(R)
    
    #Calculate U
    U = W_t.T * V.T

    #check if U is a reflection (-1)
    if round(np.linalg.det(U)) == -1 :
        Z = np.diag([1,1,-1])
        U = W_t.T * Z * V.T #reflect U so that it is no longer a reflection
        S[2] = -S[2] #S has to be negative if det is -1 because we need it in the calcualtions


    if manual == False:
        print("U = ", U)
        
        #calculate E_0 and n
        E_0 = np.linalg.norm(centered_a)**2 + np.linalg.norm(centered_b)**2

        #calculate RMSD using the formula
        RMSD = np.sqrt(1/n * (E_0 - 2 * sum(S)))
        return(RMSD)
    else:
        #Rotate B
        rotated_b = U * centered_b
        #manually calculate RMSD of a and rotated b
        man_RMSD = np.sqrt((1/n) * np.linalg.norm(centered_a - rotated_b)**2)
        return(man_RMSD)

In [6]:
a = np.matrix(
        [[ 18.92238689,  9.18841188,  8.70764463,  9.38130981,  8.53057997],
        [ 1.12391951,  0.8707568 ,  1.01214183,  0.59383894,  0.65155349],
        [ 0.46106398,  0.62858099, -0.02625641,  0.35264203,  0.53670857]], 'f')
b = np.matrix(
        [[ 1.68739355,  1.38774297,  2.1959675 ,  1.51248281,  1.70793414],
        [ 8.99726755,  8.73213223,  8.86804272,  8.31722197,  8.9924607 ],
        [ 1.1668153 ,  1.1135669 ,  1.02279055,  1.06534992,  0.54881902]], 'f')

print("RMSD using the formula:", RMSD(a, b))
print("RMSD using the rotated matrix:", RMSD(a, b, manual = True))

[2.7943044  0.3091084  0.04308294]
0.04308294
-0.04308294
U =  [[-0.16213693  0.6109726   0.77487034]
 [ 0.4972902   0.72884375 -0.4706264 ]
 [-0.8522993   0.3090295  -0.42200318]]
RMSD using the formula: 3.8760709877649164
[2.7943044  0.3091084  0.04308294]
0.04308294
-0.04308294
RMSD using the rotated matrix: 3.8760709486667966


# Part 2
## Introduction
The task is to superimpose the C-alpha coordinates of Chain A in model 0 onto the C-alpha coordinates of Chain A in model 1 in PDB file 1LCD.

This is accomplished by first extracing model 0 and 1 from the PDB file. Next, we loop through each residue in the struture and check if it is an amino acid. If it is, then we loop through all the atoms and check if the atom is a C-alpha atom. If it is, then we extract the coordinates of the atom and store it in a list. The list of coordinates is then transformed into a matrix and the matrix of coordinates is used by the previous RMSD function to find the rotation matrix and to calculate the RMSD. 

The following code performs this function:

In [1]:
import Bio.PDB as PDB

def get_coord(structure):
    '''Converts a list of residues, into a matrix of coordinates'''
    coord_list = []
    for residue in structure:
        if PDB.is_aa(residue) == True: #check if the residue is an animo acid
            for atom in residue.get_atoms():
                if atom.get_id() == "CA": #check if the atom in the residue is a calpha atom
                    coord_list.append(atom.get_coord()) #get the coridnate of the atom and append to the list
    coord_matrix = np.matrix(coord_list).T #Turn the list into a matrix and transpose it
    return coord_matrix

#create parser object to open the file
parser = pdb.PDBParser(QUIET = True)
#create the structure
structure = parser.get_structure("LAC_Repressor", "1LCD.pdb")

#Isolate chain A of structure 0 and 1 (this is an NMR structure)
chain_A_0 = structure[0]["A"] #structure 0 and chain A
chain_A_1 = structure[1]["A"] #structure 1 and chain A    

<Chain id=A>


In [40]:
X = get_coord(chain_A_0)
Y = get_coord(chain_A_1)

assert(shape(X) == shape(Y)) #check if your matrix is actually the right size

print("RMSD using the formula:", RMSD(X, Y))
print("RMSD using the rotated matrix:", RMSD(X, Y, manual = True))

U =  [[ 0.9884573  -0.11764576  0.09545439]
 [ 0.12330487  0.9908039  -0.05570936]
 [-0.08802263  0.06683634  0.9938738 ]]
RMSD using the formula: 0.7877815509379383
RMSD using the rotated matrix: 0.7877812063384226
